# דוגמה לשימוש בכלי AutoGen


## ייבוא החבילות הנדרשות


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## יצירת הלקוח

בדוגמה זו, נשתמש ב-[GitHub Models](https://aka.ms/ai-agents-beginners/github-models) לצורך גישה ל-LLM.

ה-`model` מוגדר כ-`gpt-4o-mini`. נסו לשנות את המודל למודל אחר הזמין בשוק של GitHub Models כדי לראות תוצאות שונות.

כמבחן מהיר, פשוט נריץ שאלה פשוטה - `מהי בירת צרפת`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## הגדרת הפונקציות

בדוגמה זו, נספק לסוכן גישה לכלי שהוא פונקציה עם רשימת יעדי נופש זמינים וזמינותם.

אפשר לחשוב על זה כתסריט שבו סוכן נסיעות עשוי לקבל גישה למסד נתונים של נסיעות, לדוגמה.

במהלך הדוגמה, אתם מוזמנים לנסות להגדיר פונקציות וכלים חדשים שהסוכן יכול לקרוא להם.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## הגדרת כלי הפונקציה  
כדי שהסוכן יוכל להשתמש ב-`vacation_destinations` כ-`FunctionTool`, עלינו להגדיר אותו ככזה.  

נוסיף גם תיאור לכלי, שיעזור לסוכן לזהות למה הכלי הזה משמש ביחס למשימה שהמשתמש ביקש.  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## הגדרת הסוכן

כעת נוכל ליצור את הסוכן בקוד שלמטה. אנו מגדירים את `system_message` כדי לתת לסוכן הוראות כיצד לעזור למשתמשים למצוא יעדי חופשה.

בנוסף, אנו מגדירים את הפרמטר `reflect_on_tool_use` כ-true. זה מאפשר להשתמש ב-LLM כדי לקחת את התגובה של קריאת הכלי ולשלוח את התגובה בשפה טבעית.

ניתן להגדיר את הפרמטר כ-false כדי לראות את ההבדל.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## הפעלת הסוכן

עכשיו נוכל להפעיל את הסוכן עם הודעת המשתמש הראשונית שמבקשת לצאת לטיול לטוקיו.

ניתן לשנות את יעד העיר כדי לראות כיצד הסוכן מגיב לזמינות של העיר.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
